In [1]:
#Colab
'''
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/MyDrive/aml_final
#with open("github_token.txt", "r") as f:
#  token = f.read()
#! git clone https://{token}@github.com/Tryner/aml_final.git #clone repo
%cd aml_final/
! git pull
! pip install setfit
'''

'\nfrom google.colab import drive\ndrive.mount(\'/content/drive\', force_remount=True)\n%cd /content/drive/MyDrive/aml_final\n#with open("github_token.txt", "r") as f:\n#  token = f.read()\n#! git clone https://{token}@github.com/Tryner/aml_final.git #clone repo\n%cd aml_final/\n! git pull\n! pip install setfit\n'

In [2]:
import torch
from datasets import load_dataset, Dataset
from setfit import SetFitModel, Trainer, TrainingArguments

from train.active_learning import ActiveTrainer
from train.active_learning_config import ActiveLearningConfig
from data.dataset_config import DatasetConfig
from data.load_datasets import select_dataset, load
from train.reporter import Reporter
from train.metrics import camprehesive_metrics
import time

c:\Users\UTENTE\Documents\aml_final\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
dataset_choice = select_dataset()
dataset_name, dataset = load(dataset_choice)
active_learning_config = ActiveLearningConfig(samples_per_cycle=2, unlabeled_samples=20, balancing_factor=0.5) # speed up training, not advisable
dataset_config = DatasetConfig()
train_args = TrainingArguments(num_epochs=1) #speed up training, not advisable

Select dataset:
1. sst2
2. french
3. spanish
4. indonesian
twitter_humor


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
final_reporter = Reporter("example_final_report.csv", label_column=dataset_config.label_column)
cycle_reporter = Reporter("example_cycle_report.csv", report_train_args=False, label_column=dataset_config.label_column)

def model_init():
    return SetFitModel.from_pretrained(active_learning_config.model_name, use_differentiable_head=True, head_params={"out_features": dataset_config.num_classes}).to(device)
def after_train_callback(trainer: Trainer, dataset: Dataset, run_id: int):
    cycle_reporter.report(trainer=trainer, dataset=dataset, run_id=run_id)


In [5]:

for run_id in range(2):
    trainer = ActiveTrainer(
    model_init=model_init, 
    full_train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    train_args=train_args,
    active_learning_config=active_learning_config, 
    dataset_config=dataset_config,
    after_train_callback=after_train_callback,
    metric=camprehesive_metrics
    )
    start_time=time.time()
    t = trainer.train()
    execution_time=time.time() - start_time
    final_reporter.report(
        trainer=t, 
        dataset=trainer.train_subset, 
        active_learning_config=active_learning_config, 
        dataset_name=dataset_name, run_id=run_id, execution_time=execution_time, device=device #kwars, so you can put anything here
        )


Casting the dataset: 100%|██████████| 2/2 [00:00<00:00, 323.66 examples/s]


Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Map: 100%|██████████| 2/2 [00:00<?, ? examples/s]
***** Running training *****
  Num examples = 1
  Num epochs = 1
  Total optimization steps = 1
  Total train batch size = 4
  0%|          | 0/1 [00:00<?, ?it/s]

                                          
100%|██████████| 1/1 [00:03<00:00,  3.17s/it]
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 512.


{'embedding_loss': 0.3507, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 3.2108, 'train_samples_per_second': 4.983, 'train_steps_per_second': 0.311, 'epoch': 1.0}


Epoch: 100%|██████████| 1/1 [00:04<00:00,  4.19s/it]
***** Running evaluation *****
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Map: 100%|██████████| 4/4 [00:00<00:00, 2188.52 examples/s]
***** Running training *****
  Num examples = 1
  Num epochs = 1
  Total optimization steps = 1
  Total train batch size = 14
  0%|          | 0/1 [00:00<?, ?it/s]

                                          
100%|██████████| 1/1 [00:06<00:00,  6.04s/it]
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 512.


{'embedding_loss': 0.364, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 6.0423, 'train_samples_per_second': 2.648, 'train_steps_per_second': 0.165, 'epoch': 1.0}


...

{'embedding_loss': 0.3615, 'learning_rate': 2e-05, 'epoch': 0.25}



100%|██████████| 4/4 [00:30<00:00,  7.57s/it]
The `max_length` is `None`. Using the maximum acceptable length according to the current model body: 512.


{'train_runtime': 30.3012, 'train_samples_per_second': 2.112, 'train_steps_per_second': 0.132, 'epoch': 1.0}


Epoch: 100%|██████████| 1/1 [00:23<00:00, 23.00s/it]
***** Running evaluation *****
c:\Users\UTENTE\Documents\aml_final\.venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running evaluation *****


KeyboardInterrupt: 